Installing **cleverhans** library for performing attack

In [1]:
!pip install cleverhans

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 542.5 kB/s eta 0:00:00


Importing necessary libraries used in the Notebook

In [2]:
import torch
import torchvision
from torchvision import datasets, transforms
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent
import matplotlib.pyplot as plt
import numpy as np

Initiating **cuda** GPU

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading **test images** using DataLoader with **batch_size = 1** because we want to calculate distance for each test image

In [4]:
BATCH_SIZE = 1

transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
testset = datasets.CIFAR10(root="~/data",
                           train=False,
                           transform=transform,
                           download=True)

testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True)

# Truncated testset with 1000 test images (shuffled)

truncated_testset, _ = torch.utils.data.random_split(testset, 
                                                     [1000, 9000],
                                                     generator=torch.Generator().manual_seed(0))

truncated_testloader = torch.utils.data.DataLoader(truncated_testset,
                                                   batch_size=BATCH_SIZE,
                                                   shuffle=True)

100%|██████████| 170498071/170498071 [00:13<00:00, 13103921.27it/s]


Extracting /root/data/cifar-10-python.tar.gz to /root/data


Loading the trained model (ResNet 50)

In [ ]:
PATH = "/content/main_model.pth"
resnet = torch.load(PATH)
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Projected gradient descent attack method with early stopping functionality

In [ ]:
def pgd_with_early_stopping(model, inputs, labels, eps, number_of_iters, epsilon_iters, norm):

  label_flipped = False
  
  for i in range(number_of_iters):

    adv_inputs = projected_gradient_descent(model,
                                            inputs,
                                            eps=eps,
                                            eps_iter=epsilon_iters,
                                            nb_iter=1,
                                            norm=norm,
                                            rand_init=False,
                                            sanity_checks=False)
    

    outputs = resnet(adv_inputs)

    _, predicted = torch.max(outputs.data, 1)

    label_flipped = bool(predicted!=labels)

    if label_flipped:
      print(f"Iterations for successful attack: {i}")
      break
    inputs = adv_inputs.clone()
  return adv_inputs

Clearing of CUDA Memory for efficient computation

In [ ]:
torch.cuda.empty_cache()

Defining required parameters for attack

In [ ]:
epsilon = (1e-4)/5
eps_iter = 1e-5
nb = 30
norm = np.inf

Calculating adversarial distance including misclassified inputs

In [ ]:
def adv_distance_calculation(testloader, model):  
  distance_list_0, image_idx_0 = [], []  
  distance_list_1, image_idx_1 = [], []  
  distance_list_2, image_idx_2 = [], []

  for i, data in enumerate(testloader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)

    adv_inputs = pgd_with_early_stopping(model, inputs, labels, epsilon, nb, eps_iter, norm)

    distance = torch.norm((inputs-adv_inputs), p=1)
    distance_list_0.append(distance)
    image_idx_0.append(i)

    if (predicted==labels):

      distance = torch.norm((inputs-adv_inputs), p=1)
      distance_list_1.append(distance)
      distance_list_2.append(distance)
      image_idx_1.append(i)
      image_idx_2.append(i)

    else:
      distance_list_1.append(0)
      image_idx_1.append(i)

    if i%100 == 0:
      print(f"Completed: {i}, l1_distance: {distance}")
  return distance_list_0, image_idx_0, distance_list_1, image_idx_1, distance_list_2, image_idx_2

Conversion of input_metric to its numpy type for plotting

In [ ]:
dst0, idx0, dst1, idx1, dst2, idx2 = adv_distance_calculation(truncated_testloader, resnet)

Iterations for successful attack: 14
Completed: 0, l1_distance: 15.80423355102539
Iterations for successful attack: 0
Iterations for successful attack: 1
Iterations for successful attack: 4
Iterations for successful attack: 4
Iterations for successful attack: 0
Iterations for successful attack: 1
Iterations for successful attack: 1
Iterations for successful attack: 1
Iterations for successful attack: 2
Iterations for successful attack: 5
Iterations for successful attack: 3
Iterations for successful attack: 5
Iterations for successful attack: 2
Iterations for successful attack: 3
Iterations for successful attack: 3
Iterations for successful attack: 0
Iterations for successful attack: 5
Iterations for successful attack: 3
Iterations for successful attack: 0
Iterations for successful attack: 7
Iterations for successful attack: 11
Iterations for successful attack: 3
Iterations for successful attack: 6
Iterations for successful attack: 2
Iterations for successful attack: 1
Iterations for su

In [ ]:
def conv(input_metric):
  input_metric_tensor = torch.tensor(input_metric)
  input_metric_np = input_metric_tensor.cpu().numpy()
  return input_metric_np

Visualization

In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo

distance_list_0_np = conv(dst0)
distance_list_1_np = conv(dst1)
distance_list_2_np = conv(dst2)

# Create trace objects for each line plot
trace_0 = go.Scatter(x=idx0, y=distance_list_0_np, mode='markers', name='L1 distances (including misclassified inputs)', marker=dict(color='red'))
trace_1 = go.Scatter(x=idx1, y=distance_list_1_np, mode='markers', name='L1 distances (distance=0 for misclassified inputs)', marker=dict(color='green'))
trace_2 = go.Scatter(x=idx2, y=distance_list_2_np, mode='markers', name='L1 distances (excluding for misclassified inputs)', marker=dict(color='blue'))

# Create a data list containing the trace objects
data = [trace_0, trace_1, trace_2]

# Define the layout for the plot
layout = go.Layout(title='L1 Distances for all Inputs and Misclassified Inputs', xaxis=dict(title='Image index'), yaxis=dict(title='L1 distance'))

# Create a figure object
fig = go.Figure(data=data, layout=layout)

# Display the plot in the notebook
pyo.iplot(fig)
